## Do we not need to extract route stops based on journey pattern ID and not just  LineID alone???

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### Static Variables

In [2]:
lines = ['1', '102', '104', '11', '111', '114', '116', '118', '120', '122', '123', '13', '130', '14', '140', '142', '145', '14C', '15', '150', '151', '15A', '15B', '16', '161', '16C', '17', '17A', '18', '184', '185', '220', '236', '238', '239', '25', '25A', '25B', '25X', '26', '27', '270', '27A', '27B', '27X', '29A', '31', '31A', '31B', '32', '32X', '33', '332', '33A', '33B', '33X', '37', '38', '38A', '38B', '39', '39A', '4', '40', '40B', '40D', '41', '41A', '41B', '41C', '41X', '42', '43', '44', '44B', '45A', '46A', '46E', '47', '49', '51D', '51X', '53', '54A', '56A', '59', '61', '63', '65', '65B', '66', '66A', '66B', '66X', '67', '67X', '68', '68A', '69', '69X', '7', '70', '747', '75', '76', '76A', '77A', '79', '79A', '7B', '7D', '8', '83', '83A', '84', '84A', '84X', '86', '9']

In [3]:
print(len(lines))

119


### Looped Writing of Route Stops

In [17]:
for line in lines:
    ### Reading Data

    df = pd.read_csv("bus_data/line_data/" + line + ".csv", low_memory=False, header=None)
    df.columns = ["Timestamp", "LineID", "JourneyPatternID", "TimeFrame", 
                  "VehicleJourneyID", "Lon", "Lat", "VehicleID", "StopID", 
                  "AtStop", "HumanTime", "Day", "Hour", "Runtime"]

    #convert StopID to string

    df['StopID'] = df['StopID'].astype('str')


    ### Getting Stop Locations
    # Isolating variations - getting variation with most stops

    patterns = df['JourneyPatternID'].unique()

    for pattern in patterns:
        print(pattern)

    # Concat location data

    df['location'] = ""
    df['location'] = df['Lon'].astype(str) + "_" + df['Lat'].astype(str)

    stop_loc = {}

    # Getting biggest pattern

    max_pattern = df['JourneyPatternID'].value_counts().idxmax()
    new_df = df[df.JourneyPatternID == max_pattern]

    # cutting stops that don't appear more than 20 times

    groups = new_df.groupby('StopID')
    new_df = groups.filter(lambda x: len(x) > 20)

    groups = new_df.groupby('StopID')

    # Extracting the most frequent locations for each stop
    loc_df = groups['location'].agg(lambda x:x.value_counts().index[0]).to_frame()
    loc_df.reset_index(level=0, inplace=True)

    # Converting location back to float columns
    loc_df['Lon'], loc_df['Lat'] = loc_df['location'].str.split('_', 1).str
    loc_df['Lon'], loc_df['Lat'] = loc_df['Lon'].astype('float64'), loc_df['Lat'].astype('float64')

    # drop concatenated colum 'location'
    loc_df = loc_df.drop('location', axis=1)

    ### Getting Stop Orders

    runtimes = new_df.groupby("StopID").Runtime.mean()

    order = runtimes.sort_values().to_frame()
    order.reset_index(level=0, inplace=True)
    order['Order'] = order.index

    ### Merging dataframes and saving results

    static_stops = pd.merge(order, loc_df, on='StopID')
    static_stops

    static_stops.to_csv('bus_data/static_data/routestops' + line + '.csv')

2
1002
1001
1
3
1
1002
1001
1
1001
1001
1
1002
2
1001
1
1
1001
1002
1001
1
1001
1
1001
1002
2
1
1001
1002
2
1001
1
2
1002
7
2
4
1001
1002
1004
1
3
1003
8
1007
1009
1008
1012
6
1005
1010
5
1001
1
1
1001
1003
1
1001
2
1002
1001
1
1
1001
1006
1003
1004
4
5
1007
2
1005
3
1008
1002
1001
1
1
1001
1002
2
1001
1
1002
1001
1
2
1002
1003
1001
1
2
2
1001
1
1002
3
1001
2
4
3
1
1002
1001
1
2
3
1001
1
1003
1
1002
1001
4
3
5
1004
2
6
5
1003
4
1004
1001
1
1002
2
1002
1
1001
1
1001
1001
1
4
3
1002
2
5
1007
1006
1004
6
1003
1005
1008
1001
1
2
1002
1
1001
2
1
1001
1001
1
2
1001
1
1
1001
1001
1
1001
1002
1
1001
1
1
1003
3
1002
1001
2
1001
1
1002
1001
1
2
1002
1
4
5
1006
1003
3
1004
1001
1005
1001
1
1001
1
1001
2
1
3
1001
1
1003
2
1002
1001
1
3
1001
1
1001
1002
1
2
1001
1004
1
3
1005
1002


FileNotFoundError: File b'bus_data/line_data/332.csv' does not exist